# Scrape Knowledge Terms from Wikipedia

In [40]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [33]:
url="https://en.wikipedia.org/wiki/Wikipedia:Contents/Categories"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [45]:
words = {}
words['categories'] = ['Culture and the arts','Geography and places','Health and fitness','History and events',
                       'Human activities','Mathematics and logic','Natural and physical sciences','People and self',
                       'Philosophy and thinking','Religion and belief systems','Society and social sciences',
                       'Technology and applied sciences']

In [35]:
categories = soup.find('div', attrs={"class":"contentsPage contentsPage--type"})

In [36]:
subcategories = categories.findAll('div', attrs={"class":"contentsPage__section"})

In [37]:
subcategories_words = []
for subcategory in subcategories[1:]:
    words = [subcategory_words.text for subcategory_words in subcategory.findAll('a')]
    subcategories_words = subcategories_words + words
subcategories_words = list(np.unique(np.array(subcategories_words)))[1:]
print(subcategories_words)

['Activism', 'Activists', 'Actors', 'Aerobics', 'Aerospace engineering', 'Aesthetics', 'Africa', 'Agnosticism', 'Agriculture', 'Agronomy', 'Air sports', 'Algebra', 'Allah', 'Alter egos', 'Alternative medicine', 'America', 'American football', 'Amino acids', 'Analysis of variance', 'Animals', 'Animation', 'Animism', 'Anthropology', 'Applied sciences', 'Archaeology', 'Architecture', 'Arguments', 'Arithmetic', 'Artificial intelligence', 'Arts', 'Arts and crafts', 'Asia', 'Association football', 'Astronauts', 'Astronomy', 'Atheism', 'Atmospheric sciences', 'Attention', 'Auto racing', 'Automation', 'Automobiles', 'Aviation', 'Avionics', 'Ayyavazhi', "Bahá'í Faith", 'Baseball', 'Basketball', 'Bayesian statistics', 'Beginners and newcomers', 'Belief', 'Bible', 'Billionaires', 'Bioengineering', 'Biology', 'Biotechnology', 'Board games', 'Boating', 'Bodies of water', 'Bodyweight exercise (Calisthenics)', 'Botany', 'Boxing', 'Branches', 'Broadcasting', 'Buddhas', 'Buddhism', 'Buddhist mythology'

In [38]:
subcategories_and_pages_words = []
count = 0
for word in subcategories_words:
    count+=1
    print(count/len(subcategories_words))
    url_word = 'https://en.wikipedia.org/wiki/Category:' + word
    response = requests.get(url_word)
    soup_word = BeautifulSoup(response.text, 'html.parser')
    subcategories = soup_word.findAll('div', attrs={"class":"mw-content-ltr"})
    for subcategory in subcategories[1:]:
        words = [subcategory_words.text for subcategory_words in subcategory.findAll('a')]
        subcategories_and_pages_words = subcategories_and_pages_words + words
subcategories_and_pages_words = list(np.unique(np.array(subcategories_and_pages_words)))[1:]
print(subcategories_and_pages_words)

0.0015174506828528073
0.0030349013657056147
0.004552352048558422
0.006069802731411229
0.007587253414264037
0.009104704097116844
0.010622154779969651
0.012139605462822459
0.013657056145675266
0.015174506828528073
0.01669195751138088
0.018209408194233688
0.019726858877086494
0.021244309559939303
0.02276176024279211
0.024279210925644917
0.025796661608497723
0.027314112291350532
0.028831562974203338
0.030349013657056147
0.03186646433990895
0.03338391502276176
0.03490136570561457
0.036418816388467376
0.03793626707132018
0.03945371775417299
0.0409711684370258
0.042488619119878605
0.04400606980273141
0.04552352048558422
0.04704097116843703
0.048558421851289835
0.05007587253414264
0.051593323216995446
0.05311077389984825
0.054628224582701064
0.05614567526555387
0.057663125948406675
0.05918057663125948
0.06069802731411229
0.0622154779969651
0.0637329286798179
0.06525037936267071
0.06676783004552352
0.06828528072837632
0.06980273141122914
0.07132018209408195
0.07283763277693475
0.074355083459787

0.6403641881638846
0.6418816388467374
0.6433990895295902
0.644916540212443
0.6464339908952959
0.6479514415781487
0.6494688922610015
0.6509863429438544
0.6525037936267072
0.65402124430956
0.6555386949924128
0.6570561456752656
0.6585735963581184
0.6600910470409712
0.661608497723824
0.6631259484066768
0.6646433990895296
0.6661608497723824
0.6676783004552352
0.669195751138088
0.6707132018209409
0.6722306525037937
0.6737481031866465
0.6752655538694993
0.6767830045523521
0.6783004552352049
0.6798179059180577
0.6813353566009105
0.6828528072837633
0.6843702579666161
0.6858877086494689
0.6874051593323217
0.6889226100151745
0.6904400606980273
0.6919575113808801
0.6934749620637329
0.6949924127465857
0.6965098634294385
0.6980273141122914
0.6995447647951442
0.701062215477997
0.7025796661608498
0.7040971168437026
0.7056145675265554
0.7071320182094082
0.708649468892261
0.7101669195751138
0.7116843702579666
0.7132018209408194
0.7147192716236722
0.716236722306525
0.7177541729893778
0.7192716236722306
0

In [43]:
len(subcategories_and_pages_words) 

43053

In [46]:
words['subcategories'] = subcategories_words
words['pages'] = subcategories_and_pages_words

In [48]:
words_pd = []
rank = []
for key in words.keys():
    words_pd = words_pd + words[key]
    for word in words[key]:
        rank.append(key)

# Converting terms into word vectors and perform t-SNE on them

### Convert terms into word vectors using BERT

In [57]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch
from sklearn.manifold import TSNE

In [50]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
# Getting embeddings for the target word
word_embeddings = []
count = 0

for word in words_pd:
    count+=1
    print(count/len(words_pd)*100)
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(word, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    # Find the position text in list of tokens
    term_embedding = list_token_embeddings[0]
    for word in tokenized_text[1:]:
        word_index = tokenized_text.index(word)
        word_embedding = list_token_embeddings[word_index]
        term_embedding += word_embedding
    term_embedding = list(np.array(term_embedding)/len(tokenized_text))

    word_embeddings.append(word_embedding)

0.002287073460799561
0.004574146921599122
0.0068612203823986825
0.009148293843198244
0.011435367303997804
0.013722440764797365
0.016009514225596926
0.018296587686396488
0.020583661147196047
0.02287073460799561
0.025157808068795168
0.02744488152959473
0.02973195499039429
0.03201902845119385
0.03430610191199341
0.036593175372792976
0.038880248833592534
0.04116732229439209
0.04345439575519166
0.04574146921599122
0.04802854267679078
0.050315616137590335
0.0526026895983899
0.05488976305918946
0.057176836519989026
0.05946390998078858
0.06175098344158815
0.0640380569023877
0.06632513036318727
0.06861220382398682
0.07089927728478639
0.07318635074558595
0.07547342420638552
0.07776049766718507
0.08004757112798462
0.08233464458878419
0.08462171804958375
0.08690879151038332
0.08919586497118287
0.09148293843198244
0.09377001189278199
0.09605708535358155
0.09834415881438113
0.10063123227518067
0.10291830573598024
0.1052053791967798
0.10749245265757937
0.10977952611837892
0.11206659957917849
0.114353

0.9834415881438112
0.9857286616046108
0.9880157350654104
0.9903028085262099
0.9925898819870095
0.994876955447809
0.9971640289086086
0.999451102369408
1.0017381758302075
1.0040252492910071
1.0063123227518067
1.0085993962126065
1.010886469673406
1.0131735431342055
1.015460616595005
1.0177476900558047
1.020034763516604
1.0223218369774036
1.0246089104382032
1.0268959838990028
1.0291830573598024
1.0314701308206018
1.0337572042814016
1.0360442777422012
1.0383313512030008
1.0406184246638002
1.0429054981245998
1.0451925715853994
1.047479645046199
1.0497667185069983
1.052053791967798
1.0543408654285975
1.056627938889397
1.0589150123501967
1.0612020858109963
1.0634891592717959
1.0657762327325955
1.0680633061933948
1.0703503796541944
1.072637453114994
1.0749245265757936
1.0772116000365932
1.0794986734973926
1.0817857469581922
1.084072820418992
1.0863598938797914
1.088646967340591
1.0909340408013906
1.0932211142621902
1.0955081877229897
1.0977952611837891
1.1000823346445887
1.1023694081053883
1.10

1.985179763974019
1.9874668374348183
1.989753910895618
1.9920409843564177
1.9943280578172171
1.9966151312780167
1.998902204738816
2.001189278199616
2.003476351660415
2.005763425121215
2.0080504985820142
2.010337572042814
2.0126246455036134
2.014911718964413
2.017198792425213
2.019485865886012
2.021772939346812
2.0240600128076114
2.026347086268411
2.0286341597292106
2.03092123319001
2.0332083066508093
2.0354953801116094
2.0377824535724085
2.040069527033208
2.042356600494008
2.0446436739548073
2.0469307474156073
2.0492178208764065
2.051504894337206
2.0537919677980057
2.0560790412588053
2.058366114719605
2.0606531881804044
2.0629402616412036
2.0652273351020036
2.067514408562803
2.0698014820236024
2.0720885554844024
2.0743756289452016
2.0766627024060016
2.0789497758668007
2.0812368493276003
2.0835239227884
2.0858109962491995
2.088098069709999
2.0903851431707987
2.0926722166315983
2.094959290092398
2.0972463635531975
2.0995334370139966
2.1018205104747967
2.104107583935596
2.1063946573963954

3.009788674412222
3.0120757478730216
3.0143628213338216
3.0166498947946208
3.0189369682554204
3.02122404171622
3.0235111151770195
3.025798188637819
3.0280852620986187
3.030372335559418
3.032659409020218
3.0349464824810175
3.0372335559418167
3.0395206294026167
3.041807702863416
3.0440947763242154
3.046381849785015
3.0486689232458146
3.050955996706614
3.053243070167414
3.0555301436282134
3.057817217089013
3.0601042905498126
3.062391364010612
3.0646784374714118
3.066965510932211
3.069252584393011
3.07153965785381
3.0738267313146097
3.0761138047754093
3.078400878236209
3.0806879516970085
3.082975025157808
3.0852620986186077
3.0875491720794073
3.089836245540207
3.0921233190010065
3.094410392461806
3.096697465922605
3.0989845393834052
3.1012716128442044
3.103558686305004
3.1058457597658036
3.108132833226603
3.1104199066874028
3.1127069801482024
3.114994053609002
3.1172811270698015
3.119568200530601
3.1218552739914007
3.1241423474522003
3.1264294209129995
3.128716494373799
3.131003567834599
3

4.025249291007227
4.027536364468027
4.029823437928826
4.032110511389626
4.034397584850426
4.036684658311225
4.038971731772024
4.0412588052328235
4.043545878693624
4.045832952154424
4.048120025615223
4.050407099076022
4.052694172536822
4.054981245997621
4.057268319458421
4.059555392919221
4.06184246638002
4.0641295398408195
4.066416613301619
4.068703686762419
4.070990760223219
4.073277833684018
4.075564907144817
4.077851980605617
4.080139054066416
4.082426127527216
4.084713200988016
4.087000274448815
4.089287347909615
4.091574421370414
4.093861494831215
4.096148568292014
4.098435641752813
4.100722715213612
4.103009788674412
4.105296862135212
4.107583935596011
4.109871009056811
4.1121580825176105
4.11444515597841
4.11673222943921
4.11901930290001
4.121306376360809
4.123593449821608
4.125880523282407
4.128167596743207
4.130454670204007
4.132741743664806
4.135028817125606
4.137315890586406
4.139602964047205
4.141890037508005
4.144177110968805
4.146464184429604
4.148751257890403
4.151038331

...

95.17884914463453
95.18113621809533
95.18342329155612
95.18571036501693
95.18799743847772
95.19028451193853
95.19257158539932
95.19485865886013
95.19714573232092
95.19943280578173
95.20171987924252
95.20400695270332
95.20629402616412
95.20858109962492
95.21086817308571
95.21315524654652
95.21544232000731
95.21772939346812
95.22001646692891
95.22230354038972
95.22459061385051
95.22687768731132
95.22916476077211
95.23145183423291
95.2337389076937
95.23602598115451
95.2383130546153
95.24060012807611
95.24288720153692
95.24517427499771
95.24746134845851
95.2497484219193
95.25203549538011
95.2543225688409
95.25660964230171
95.2588967157625
95.26118378922331
95.2634708626841
95.26575793614491
95.2680450096057
95.2703320830665
95.2726191565273
95.2749062299881
95.27719330344891
95.2794803769097
95.28176745037051
95.2840545238313
95.28634159729211
95.2886286707529
95.2909157442137
95.29320281767451
95.2954898911353
95.29777696459611
95.3000640380569
95.30235111151771
95.3046381849785
95.306925

96.22861586314151
96.23090293660232
96.23319001006311
96.23547708352392
96.23776415698472
96.24005123044552
96.24233830390632
96.24462537736711
96.24691245082792
96.24919952428871
96.25148659774952
96.25377367121033
96.25606074467112
96.25834781813192
96.26063489159272
96.26292196505352
96.26520903851431
96.26749611197512
96.26978318543593
96.27207025889672
96.27435733235752
96.27664440581832
96.27893147927912
96.28121855273992
96.28350562620072
96.28579269966151
96.28807977312232
96.29036684658311
96.29265392004392
96.29494099350471
96.29722806696552
96.29951514042631
96.30180221388711
96.3040892873479
96.30637636080871
96.3086634342695
96.31095050773031
96.3132375811911
96.31552465465191
96.31781172811272
96.32009880157351
96.32238587503431
96.3246729484951
96.32696002195591
96.3292470954167
96.33153416887751
96.3338212423383
96.33610831579911
96.3383953892599
96.34068246272071
96.3429695361815
96.3452566096423
96.3475436831031
96.3498307565639
96.3521178300247
96.3544049034855
96.35

97.28753087549173
97.28981794895252
97.29210502241332
97.29439209587412
97.29667916933492
97.29896624279571
97.30125331625652
97.30354038971731
97.30582746317812
97.30811453663891
97.31040161009972
97.31268868356051
97.31497575702132
97.31726283048211
97.31954990394291
97.32183697740372
97.32412405086451
97.32641112432532
97.32869819778611
97.33098527124692
97.33327234470771
97.33555941816851
97.3378464916293
97.34013356509011
97.3424206385509
97.34470771201171
97.3469947854725
97.34928185893331
97.3515689323941
97.35385600585491
97.3561430793157
97.3584301527765
97.3607172262373
97.3630042996981
97.3652913731589
97.3675784466197
97.3698655200805
97.3721525935413
97.3744396670021
97.3767267404629
97.37901381392369
97.3813008873845
97.38358796084529
97.3858750343061
97.3881621077669
97.3904491812277
97.3927362546885
97.39502332814929
97.3973104016101
97.39959747507089
97.4018845485317
97.4041716219925
97.4064586954533
97.4087457689141
97.4110328423749
97.4133199158357
97.41560698929649


98.34644588784191
98.34873296130272
98.35102003476351
98.35330710822431
98.35559418168512
98.35788125514591
98.36016832860672
98.36245540206751
98.36474247552832
98.36702954898911
98.36931662244992
98.37160369591072
98.37389076937151
98.37617784283232
98.37846491629311
98.38075198975392
98.38303906321471
98.38532613667552
98.38761321013631
98.38990028359711
98.39218735705792
98.39447443051871
98.39676150397952
98.39904857744031
98.40133565090112
98.40362272436191
98.40590979782272
98.40819687128351
98.41048394474431
98.4127710182051
98.41505809166591
98.4173451651267
98.41963223858751
98.4219193120483
98.42420638550911
98.4264934589699
98.42878053243071
98.4310676058915
98.4333546793523
98.4356417528131
98.4379288262739
98.4402158997347
98.4425029731955
98.4447900466563
98.4470771201171
98.44936419357789
98.4516512670387
98.45393834049949
98.4562254139603
98.4585124874211
98.4607995608819
98.4630866343427
98.4653737078035
98.4676607812643
98.46994785472509
98.4722349281859
98.474522001

99.40307382673132
99.40536090019212
99.40764797365291
99.40993504711372
99.41222212057451
99.41450919403532
99.41679626749611
99.41908334095692
99.42137041441771
99.42365748787851
99.4259445613393
99.42823163480011
99.4305187082609
99.43280578172171
99.4350928551825
99.43737992864331
99.4396670021041
99.44195407556491
99.4442411490257
99.4465282224865
99.4488152959473
99.4511023694081
99.4533894428689
99.4556765163297
99.4579635897905
99.4602506632513
99.4625377367121
99.4648248101729
99.4671118836337
99.4693989570945
99.4716860305553
99.4739731040161
99.4762601774769
99.4785472509377
99.4808343243985
99.48312139785929
99.4854084713201
99.48769554478089
99.4899826182417
99.49226969170249
99.4945567651633
99.49684383862409
99.4991309120849
99.50141798554569
99.50370505900649
99.5059921324673
99.50827920592809
99.5105662793889
99.51285335284969
99.5151404263105
99.51742749977129
99.5197145732321
99.5220016466929
99.52428872015369
99.5265757936145
99.52886286707529
99.5311499405361
99.533

### Perform t-SNE algorithm on the word vectors to turn them into 3d

In [59]:
word_embeddings_3d = TSNE(n_components=3, learning_rate='auto',
                  init='random').fit_transform(word_embeddings)
word_embeddings_3d.shape

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

### Save all information to a csv file

In [62]:
df = pd.DataFrame(list(zip(words_pd, rank, word_embeddings))
                  , columns =['term', 'rank', 'embedding'])
df.to_csv("wiki_terms.csv",index=False)